# Desafío N° 2 
## 5 - Machine Learning

### Grupo N° 5

    Integrantes:
                Arangue, Marcelo            
                Bardauil, Joaquín              
                Marquez, Hector              
                Neustadt, Alejandro       
                Pero, Felipe                                    

In [1]:
%matplotlib inline
import random
import numpy as np
from matplotlib import pyplot as plt
from sklearn import datasets, linear_model

import pandas as pd
from scipy import stats
import seaborn as sns
from sklearn import datasets
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df_dpto_connulos = pd.read_csv("prop_apartament_dummies.csv", usecols=[7,8,10,23,24,26,27,35])
df_dpto_connulos.columns

Index(['surface_covered_in_m2', 'rooms', 'price_aprox_usd', 'balcon',
       'centrico', 'torre', 'Cochera', 'Amenities'],
      dtype='object')

In [3]:
df_dpto = df_dpto_connulos.dropna()
df_dpto.shape

(38187, 8)

In [4]:
df_dpto['surface_covered_in_m2 * rooms'] = df_dpto['surface_covered_in_m2'] * df_dpto['rooms']

C:\Users\aneustadt\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [5]:
df_dpto.columns

Index(['surface_covered_in_m2', 'rooms', 'price_aprox_usd', 'balcon',
       'centrico', 'torre', 'Cochera', 'Amenities',
       'surface_covered_in_m2 * rooms'],
      dtype='object')

In [6]:
X = df_dpto[['surface_covered_in_m2', 'rooms','surface_covered_in_m2 * rooms','balcon','centrico','torre','Cochera','Amenities']]
y = df_dpto.price_aprox_usd

Tengo que elegir entre los tres modelos(regresión lineal, regresion regularizada Ridgea y regresio regularizada LASSO)

In [7]:
# Primero, dividimos el dataser en training/validacion y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

# habia pensado en separar el Xtest e ytest, pero como la funcion esta inicializada siempre da el mismo random, entonces
# no mete el test en el training generando overfiting
X_test.head()

(30549, 8) (30549,)
(7638, 8) (7638,)


,surface_covered_in_m2,rooms,surface_covered_in_m2 * rooms,balcon,centrico,torre,Cochera,Amenities
29714,41.0,1.0,41.0,0,0,0,0,1
37420,44.0,1.0,44.0,0,0,0,0,0
32184,158.0,4.0,632.0,1,0,0,1,0
38103,46.0,2.0,92.0,0,1,0,0,1
7713,130.0,4.0,520.0,1,0,0,1,1


In [8]:
# Ahora bien, vamos a realizar el proceso de tunning de los hiperparámetros de los modelos en base a validación cruzada.
# Por eso, instanciamos previamente tres objetos:
al_ridge = np.linspace(0.001, 0.5, 500)  
al_lasso = np.linspace(0.001, 0.5, 500)
kf = KFold(n_splits=5, shuffle=True)

In [9]:
# Instanciamos los modelos:
lm = LinearRegression()
lm_ridge_cv= RidgeCV(alphas=al_ridge, cv=kf, normalize=True)
lm_lasso_cv = LassoCV(alphas=al_lasso, cv=kf, normalize=True)
elastic_net = linear_model.ElasticNet(alpha=0.5, normalize=True)

In [10]:
# Hacemos los fits respectivos
lm.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [11]:
lm_ridge_cv.fit(X_train, y_train)

RidgeCV(alphas=array([0.001, 0.002, ..., 0.499, 0.5  ]),
    cv=KFold(n_splits=5, random_state=None, shuffle=True),
    fit_intercept=True, gcv_mode=None, normalize=True, scoring=None,
    store_cv_values=False)

In [12]:
lm_lasso_cv.fit(X_train, y_train)

LassoCV(alphas=array([0.001, 0.002, ..., 0.499, 0.5  ]), copy_X=True,
    cv=KFold(n_splits=5, random_state=None, shuffle=True), eps=0.001,
    fit_intercept=True, max_iter=1000, n_alphas=100, n_jobs=None,
    normalize=True, positive=False, precompute='auto', random_state=None,
    selection='cyclic', tol=0.0001, verbose=False)

In [13]:
elastic_net.fit(X_train, y_train)

ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=True, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [14]:
# Calculamos el R2 y el RMSE para ver la performance tiene cada modelo en el Training Set 

print(" Score Train Lineal: %.2f\n" % lm.score(X_train, y_train),
      "Score Train Ridge : %.2f\n" % lm_ridge_cv.score(X_train, y_train),
      "Score Train Lasso : %.2f\n" %  lm_lasso_cv.score(X_train, y_train),
      "Score Train Elastic : %.2f\n" % elastic_net.score(X_train, y_train))

y_pred_tr_lm = lm.predict(X_train)
y_pred_tr_ridge = lm_ridge_cv.predict(X_train)
y_pred_tr_lasso = lm_lasso_cv.predict(X_train)
y_pred_tr_elast = elastic_net.predict(X_train)

rmse = lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred))

print(" Train RMSE lineal   : %.2f \n" % rmse(y_train,y_pred_tr_lm),
      "Train RMSE Ridge    : %.2f \n" % rmse(y_train,y_pred_tr_ridge),
      "Train RMSE Lasso    : %.2f \n" % rmse(y_train,y_pred_tr_lasso),
     "Train RMSE Elastic    : %.2f \n" % rmse(y_train,y_pred_tr_elast))

 Score Train Lineal: 0.74
 Score Train Ridge : 0.74
 Score Train Lasso : 0.74
 Score Train Elastic : 0.00

 Train RMSE lineal   : 77644.64 
 Train RMSE Ridge    : 77645.01 
 Train RMSE Lasso    : 77644.65 
 Train RMSE Elastic    : 152085.93 



In [15]:
# Lo único que queda es evaluar la performance del modelo. 
# Para ello debemos usar datos que no hayan formado parte del proceso de entrenamiento anterior.
# Entonces usamos el Test Set. 
# Hacemos las predicciones sobre la matriz de predictores del Test Set

y_pred_lm = lm.predict(X_test)
y_pred_ridge = lm_ridge_cv.predict(X_test)
y_pred_lasso = lm_lasso_cv.predict(X_test)
y_pred_elast = elastic_net.predict(X_test)

print(" Score Test Lineal: %.2f\n" % lm.score(X_test, y_test),
      "Score Test Ridge : %.2f\n" % lm_ridge_cv.score(X_test, y_test),
      "Score Test Lasso : %.2f\n" %  lm_lasso_cv.score(X_test, y_test),
      "Score Test Elastic : %.2f\n" % elastic_net.score(X_test, y_test))

# Testeo final del modelo sobre Test Set

print(" Test RMSE lineal= %.2f\n" % rmse(y_test, y_pred_lm),
      "Test RMSE Ridge = %.2f\n" %  rmse(y_test, y_pred_ridge),
      "Test RMSE Lasso = %.2f\n" %  rmse(y_test, y_pred_lasso),
     "Test RMSE Elastic = %.2f\n" % rmse(y_test,y_pred_elast))

 Score Test Lineal: 0.69
 Score Test Ridge : 0.69
 Score Test Lasso : 0.69
 Score Test Elastic : -0.00

 Test RMSE lineal= 76687.12
 Test RMSE Ridge = 76685.34
 Test RMSE Lasso = 76685.82
 Test RMSE Elastic = 137357.83



Para saber con cual me quedo, comparo los Score Train y Test, con sus respectivos RMSE

## Regularización: Lasso

In [16]:
model = linear_model.LassoCV(cv=3).fit(X_train, y_train)
scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='r2')
dict(alpha=model.alpha_, scores=scores, mean_score=scores.mean(), zero_coefs=(model.coef_ == 0).sum())

{'alpha': 30420.936233053784,
 'scores': array([0.73055649, 0.70592506, 0.72014234, 0.72539332, 0.71011636]),
 'mean_score': 0.7184267135361283,
 'zero_coefs': 6}

In [17]:
model = linear_model.LassoCV(cv=3, normalize=True).fit(X, y)
scores = cross_val_score(model, X, y, cv=kf, scoring='r2')
dict(alpha=model.alpha_, scores=scores, mean_score=scores.mean(), zero_coefs=(model.coef_ == 0).sum())

{'alpha': 0.6447417603484233,
 'scores': array([0.73594607, 0.73436481, 0.7336515 , 0.70395978, 0.73907445]),
 'mean_score': 0.7293993204797456,
 'zero_coefs': 0}

In [18]:
model = linear_model.RidgeCV(normalize=True).fit(X, y)
scores = cross_val_score(model, X, y, cv=kf, scoring='r2')
dict(alpha=model.alpha_, scores=scores, mean_score=scores.mean())

{'alpha': 0.1,
 'scores': array([0.71312572, 0.73243474, 0.71624748, 0.71420711, 0.72268136]),
 'mean_score': 0.7197392815281904}